In [1]:
## NowPlaying Cultural Sentiment Data
"""
cs_header = ["hashtag", "AFINN_min", "AFINN_max", "AFINN_sum", "AFINN_avg", "OpinionLex_min", "OpinionLex_max", "OpinionLex_sum", "OpinionLex_avg", "Sentistrength_min", "Sentistrength_max", "Sentistrength_sum", "Sentistrength_avg", "Vader_min", "Vader_max", "Vader_sum", "Vader_avg", "SentimentHashtag_min", "SentimentHashtag_max", "SentimentHashtag_sum", "SentimentHashtag_avg"]
np_cs = pd.read_csv('np_cultural_sentiment.csv',names = cs_header)
"""
#IMPORTING Cultural Sentiment Data to MongoDB
"""
np_cs_json = json.loads(np_cs.to_json(orient="records"))
db.cultural_sentiment.drop()
db.cultural_sentiment.insert_many(np_cs_json)
"""
## Transformation of NowPlaying Cultural Tweets to DataframeStructure
"""
output = []
with open("np_cultural.json") as f:
    for line in f:     
        output.append(json.load|s(line))
np_c = pd.DataFrame(output)
"""
#IMPORTING Cultural Tweets to MongoDB
"""
db.cultural_tweet.drop()
records = json.loads(np_c.T.to_json()).values()
db.cultural_tweet.insert_many(records)
"""

## Transformation and Importing of Playlist Data from NowPlaying to MongoDB
"""
np_pl = pd.read_csv("playlist.csv", names=["playlist_id","track_id","playlist_name"])
np_pl_json = json.loads(np_pl.to_json(orient="records"))
db.playlist.drop()
db.playlist.insert_many(np_pl_json)
"""

'\nnp_pl = pd.read_csv("playlist.csv", names=["playlist_id","track_id","playlist_name"])\nnp_pl_json = json.loads(np_pl.to_json(orient="records"))\ndb.playlist.drop()\ndb.playlist.insert_many(np_pl_json)\n'

# CULTURAL #NOWPLAYING DATA
*********************************

This dataset contains data describing the listening events of users (extracted from the #nowplaying dataset), the emotion extracted from the hashtags used in the according tweets and information about the location of the user.


TWITTER AND TRACK DATA
-------------------------------
The data regarding the listening events is contained in np_cultural.zip and is encoded as json. It holds the following information:

- id: the id of the underlying tweet [*]
- user_id: the id of the user who sent the tweet (MD5 of it)
- user_lang: The BCP 47 code for the user’s self-declared user interface language. [*]
- user_time_zone: [*]
- text: actual content of the tweet [*]
- tweet_lang: language of the tweet (as detected by Twitter; BCP 47 language identifier corresponding to the machine-detected language of the Tweet text, or und if no language could be detected.) [*]
- geo: Deprecated version of coordinates (however, we deal with data stemming from before this API change, therefore we still include it; cf. coordinates for a description)
- coordinates: Represents the geographic location of this Tweet as reported by the user or client application. The inner coordinates array is formatted as geoJSON (longitude first, then latitude). [*]
- place: When present, indicates that the tweet is associated (but not necessarily originating from) a Place. [*]
- created_at: time the tweet was sent. [*]
- source: Utility used to post the Tweet, as an HTML-formatted string. [*]
- track_title: title of the track the user tweeted about
- track_id: the unique id of the track (from #nowplaying dataset) 
- artist_name: name of artist performing the track
- artist_id: the unique id of the artist (from #nowplaying dataset) 
- hashtags: list of hashtags used in the tweet.

[*] for further information about the information gathered from Twitter, please consult https://dev.twitter.com/overview/api/tweets

Please note that we do only add key-value pairs for geo/coordinates/place information if this information was provided by the Twitter API (i.e., missing keys signal that this information is not available for the given tweet).


SENTIMENT DATA
-------------------------------
The data regarding the hashtag's sentiment (if any could be obtained) is contained in np_cultural_sentiment.csv and is formatted as csv. The sentiment score was obtained by applying a set of well-known sentiment dictionaries. The sentiment scores are scaled between 0 and 1 (very negative to very positive). For each dictionary, we list the minimum, maximum, sum and average sentiment score across all hashtags used within every tweet (most tweets only feature a single hashtag we can assign a sentiment value to)
It contains the following information (in this particular order):
- name of the hashtag
- AFINN dictionary (min, max, sum, avg)
- Opinion Lexicon (min, max, sum, avg)
- Sentistrength Lexicon (min, max, sum, avg)
- Vader (min, max, sum, avg)
- Sentiment Hashtag Lexicon (min, max, sum, avg)


Please note that we only added hashtags for which we could obtain a sentiment value from at least one sentiment dictionary.

In [1]:
import pymongo
from pymongo import MongoClient
import pandas as pd
import numpy as np 
import json
import csv
import pprint
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from difflib import SequenceMatcher


In [2]:
#Call Mongo Instance
client = MongoClient()
db = client.now_playing

#Spotify Instance
client_credentials_manager = SpotifyClientCredentials(client_id="7736d10450e04c5f9e302bb07a4f6cf7", client_secret="a11e29bc5c324ddeb19fc6249d303814",)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

### Importing the MusicBrainz Mapping of artists and tracks and tracks from cultural dataset

`mongoimport -d now_playing -c mb_tracks --type csv --file mb_tracks.csv --fields "np_id,mb_track_id"`

`mongoimport -d now_playing -c mb_artists --type csv --file mb_artists.csv --fields "np_id,mb_artist_id"`

`mongoimport -d now_playing -c np_cultural_mb_tracks --type csv --file np_cultural_mb_tracks.csv --headerline
`

In [3]:
user_ids = db.cultural_tweet.distinct("user_id")
type(user_ids)


list

In [34]:
#db.cultural_tweet.find({"hashtag": "nobeats"})
user_tweets = db.cultural_tweet.find({"user_id": '1c10f9788fdcc4baf6cf6a2631fe78bc12102418'})
#TOTAL Number of Tweets
print("Total number of Tweets: "+ str(db.cultural_tweet.find({}).count()))
print("Total number of Songs: " + str(len(db.cultural_tweet.distinct("track_title"))))
#Get Distinct User Ids
print("Total number of Users: " + str(len(user_ids)))
print("Total number of Tweets with duration: "+ str(db.cultural_tweet.find({"duration_ms" : {"$exists": True}}).count()))

print("Total number of Tweets without duration: "+ str(db.cultural_tweet.find({"duration_ms" : {"$exists": False}}).count()))

Total number of Tweets: 564301
Total number of Songs: 39776
Total number of Users: 9431
Total number of Tweets with duration: 276883
Total number of Tweets without duration: 287418


In [5]:
first = user_ids[0]
user_songs = db.cultural_tweet.find({"user_id": first})
initial_song = user_songs[0]
initial_date = initial_song["created_at"]

In [6]:
#db.mb_tracks.find({"np_id": initial_song["track_id"]}).count()
#for song in user_songs:
#    print(str(song["track_title"]))
#    print(str(song["track_id"]))
    
print(len(db.cultural_tweet.distinct("track_id")))
print(len(db.cultural_tweet.distinct("track_title")))

51045
39776


In [7]:
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()
def convertMillis(millis):
     seconds = (millis/1000)%60
     minutes = (millis/(1000*60))%60
     return str(int(minutes)) + ":" +str(int(seconds))


In [8]:
"""
search_query = initial_song["track_title"].lower() +" "+ initial_song["artist_name"].lower()
results = sp.search(q=search_query,limit=5)
print("Looking for: "+search_query)
print("Querying...")
for track in results["tracks"]["items"]:
    artist_names = list(map(lambda x: x["name"],track["artists"]))
    print(" ".join(artist_names))
    print("Found: " + str(track["name"]) + " "+str(track["artists"][0]["name"]))
    print(convertMillis(track["duration_ms"]))
    result_matcher = track["name"].lower() + " " + track["artists"][0]["name"].lower()
    print("Similarity Score: " + str((similar(search_query, result_matcher))))
""

'\nsearch_query = initial_song["track_title"].lower() +" "+ initial_song["artist_name"].lower()\nresults = sp.search(q=search_query,limit=5)\nprint("Looking for: "+search_query)\nprint("Querying...")\nfor track in results["tracks"]["items"]:\n    artist_names = list(map(lambda x: x["name"],track["artists"]))\n    print(" ".join(artist_names))\n    print("Found: " + str(track["name"]) + " "+str(track["artists"][0]["name"]))\n    print(convertMillis(track["duration_ms"]))\n    result_matcher = track["name"].lower() + " " + track["artists"][0]["name"].lower()\n    print("Similarity Score: " + str((similar(search_query, result_matcher))))\n'

In [24]:
final_result = dict()
score_result = dict()
name_found_result = dict()
searched_name = dict()
#Get Spotify's song duration
track_list = db.cultural_tweet.find({"duration_ms" : {"$exists" : False }}).distinct("track_id")
track_list_len = len(track_list)
searched = 0
for track in track_list:
    song_name = db.cultural_tweet.find_one({"track_id": track }, {"track_title" : 1 , "artist_name": 1})
    search_query = song_name["track_title"].lower() + " " + song_name["artist_name"].lower()
    search_result = sp.search(q = search_query, limit= 3, type="track")
    searched += 1
    score_result[track] = 0
    for t in search_result["tracks"]["items"]:
        artists = " ".join(list(map(lambda x: x["name"],t["artists"])))
        result_matcher = t["name"].lower() + " " + artists.lower()
        similarity_score = similar(search_query,result_matcher)
        if(similarity_score > score_result[track]):
            print("Found " + result_matcher + "  => Similarity Score: " + str(similarity_score))
            final_result[track] = t["duration_ms"]
            score_result[track] = similarity_score
            name_found_result[track] = result_matcher
            searched_name[track] = search_query
            break
    print("Completed " + str(searched) +" out of " + str(track_list_len))         


Completed 1 out of 44828
Found what a feeling (flashdance) - clubhouse album mix global deejays  => Similarity Score: 0.623655913978
Completed 2 out of 44828
Found bat country avenged sevenfold  => Similarity Score: 0.277777777778
Completed 3 out of 44828
Completed 4 out of 44828
Found undone - the sweater song weezer  => Similarity Score: 0.577777777778
Completed 5 out of 44828
Found under your spell the birthday massacre  => Similarity Score: 0.691358024691
Completed 6 out of 44828
Found ooh la la (from "the smurfs 2") britney spears  => Similarity Score: 0.685714285714
Completed 7 out of 44828
Found gold digger kanye west jamie foxx  => Similarity Score: 0.8
Completed 8 out of 44828
Found k.y.h.u. - noisecontrollers rmx wildstylez  => Similarity Score: 0.526315789474
Completed 9 out of 44828
Found junk of the heart (happy) the kooks  => Similarity Score: 0.870967741935
Completed 10 out of 44828
Found international love pitbull chris brown  => Similarity Score: 0.8125
Completed 11 ou

Found pressure (feat. jay-z) - explicit album version lupe fiasco jay z  => Similarity Score: 0.115942028986
Completed 106 out of 44828
Found glory to glory to glory fred hammond radical for christ  => Similarity Score: 0.791208791209
Completed 107 out of 44828
Found the way i are (dance with somebody) [feat. lil wayne] bebe rexha lil wayne  => Similarity Score: 0.258823529412
Completed 108 out of 44828
Found love galore sza travis scott  => Similarity Score: 0.432432432432
Completed 109 out of 44828
Found ruff stuff - original mix justin martin  => Similarity Score: 0.761904761905
Completed 110 out of 44828
Completed 111 out of 44828
Found vodka korpiklaani  => Similarity Score: 0.7
Completed 112 out of 44828
Found make your move - radio edit dave armstrong  => Similarity Score: 0.816901408451
Completed 113 out of 44828
Found twilight of the thunder god amon amarth  => Similarity Score: 0.516129032258
Completed 114 out of 44828
Found back in time - featured in "men in black 3" pitbull

Found airplanes (feat. hayley williams of paramore) - explicit album version b.o.b hayley williams  => Similarity Score: 0.0808080808081
Completed 197 out of 44828
Found tunnel vision kodak black  => Similarity Score: 0.421052631579
Completed 198 out of 44828
Completed 199 out of 44828
Completed 200 out of 44828
Completed 201 out of 44828
Found me without you ultimate party jams  => Similarity Score: 0.622950819672
Completed 202 out of 44828
Found lemme see usher rick ross  => Similarity Score: 0.863636363636
Completed 203 out of 44828
Found or nah (feat. the weeknd, wiz khalifa and dj mustard) - remix ty dolla $ign dj mustard wiz khalifa the weeknd  => Similarity Score: 0.31007751938
Completed 204 out of 44828
Completed 205 out of 44828
Found carry that weight - remastered the beatles  => Similarity Score: 0.816901408451
Completed 206 out of 44828
Found the last embrace spirit caravan  => Similarity Score: 0.666666666667
Completed 207 out of 44828
Found through the night - original mi

Found neighborhood #2 (laika) arcade fire  => Similarity Score: 0.653846153846
Completed 288 out of 44828
Found representin ludacris kelly rowland  => Similarity Score: 0.740740740741
Completed 289 out of 44828
Found rock n roll train ac/dc  => Similarity Score: 0.380952380952
Completed 290 out of 44828
Completed 291 out of 44828
Found all of you julio iglesias diana ross  => Similarity Score: 0.819672131148
Completed 292 out of 44828
Found everybody wants some!! - 2015 remastered van halen  => Similarity Score: 0.75
Completed 293 out of 44828
Found last man standing hammerfall  => Similarity Score: 0.571428571429
Completed 294 out of 44828
Completed 295 out of 44828
Found fake it seether  => Similarity Score: 0.666666666667
Completed 296 out of 44828
Found the first noel brian mcknight claude mcknight tim miner  => Similarity Score: 0.690476190476
Completed 297 out of 44828
Found a sunday kind of love - single version etta james  => Similarity Score: 0.3125
Completed 298 out of 44828


Completed 383 out of 44828
Found [my baby does] good sculptures the rezillos  => Similarity Score: 0.788732394366
Completed 384 out of 44828
Completed 385 out of 44828
Completed 386 out of 44828
Found hoy tengo ganas de ti alejandro fernandez christina aguilera  => Similarity Score: 0.659340659341
Completed 387 out of 44828
Found diamonds - remix rihanna kanye west  => Similarity Score: 0.703703703704
Completed 388 out of 44828
Completed 389 out of 44828
Completed 390 out of 44828
Completed 391 out of 44828
Found i will be found (lost at sea) john mayer  => Similarity Score: 0.388059701493
Completed 392 out of 44828
Completed 393 out of 44828
Completed 394 out of 44828
Found scatman (ski-ba-bop-ba-dop-bop) scatman john  => Similarity Score: 0.625
Completed 395 out of 44828
Found satisfaction benny benassi the biz  => Similarity Score: 0.866666666667
Completed 396 out of 44828
Found you're a lie slash myles kennedy the conspirators  => Similarity Score: 0.515151515152
Completed 397 out 

...

Found paranoia brain police  => Similarity Score: 1.0
Completed 13000 out of 44828
Found the man who loved life (live) the jayhawks  => Similarity Score: 0.909090909091
Completed 13001 out of 44828
Found tapha niang toumani diabaté's symmetric orchestra  => Similarity Score: 0.153846153846
Completed 13002 out of 44828
Found raining blood slayer  => Similarity Score: 0.780487804878
Completed 13003 out of 44828
Found strange are the people ape machine  => Similarity Score: 1.0
Completed 13004 out of 44828
Found infinitesimal mother mother  => Similarity Score: 1.0
Completed 13005 out of 44828
Found shelter porter robinson madeon  => Similarity Score: 0.488888888889
Completed 13006 out of 44828
Found let's do it (do it together) the chambers brothers  => Similarity Score: 0.780487804878
Completed 13007 out of 44828
Completed 13008 out of 44828
Found be my light, be my guide - live gene  => Similarity Score: 0.892307692308
Completed 13009 out of 44828
Found endless agony black pyramid  => 

Found brainless eminem  => Similarity Score: 1.0
Completed 13088 out of 44828
Found scattered diamonds hungry kids of hungary  => Similarity Score: 1.0
Completed 13089 out of 44828
Completed 13090 out of 44828
Found unfinished sympathy - 2012 mix/master massive attack  => Similarity Score: 0.790697674419
Completed 13091 out of 44828
Found nature's predators kylesa  => Similarity Score: 0.979591836735
Completed 13092 out of 44828
Completed 13093 out of 44828
Found closure owl eyes  => Similarity Score: 1.0
Completed 13094 out of 44828
Found how i roll - original mix savant  => Similarity Score: 0.69387755102
Completed 13095 out of 44828
Found jump to the beat stacy lattisaw  => Similarity Score: 1.0
Completed 13096 out of 44828
Found hypnotic syleena johnson r. kelly fabolous  => Similarity Score: 0.727272727273
Completed 13097 out of 44828
Found battle scars lupe fiasco guy sebastian  => Similarity Score: 0.774193548387
Completed 13098 out of 44828
Found hold on, we're going home drake

Completed 13184 out of 44828
Found went to poland - club mix edit bacefook  => Similarity Score: 0.741935483871
Completed 13185 out of 44828
Found paradise coldplay  => Similarity Score: 0.486486486486
Completed 13186 out of 44828
Completed 13187 out of 44828
Found away from here - us version the enemy uk  => Similarity Score: 0.75
Completed 13188 out of 44828
Found as the crow flies aynsley lister  => Similarity Score: 1.0
Completed 13189 out of 44828
Found no easy way out robert tepper  => Similarity Score: 1.0
Completed 13190 out of 44828
Found we made you eminem  => Similarity Score: 1.0
Completed 13191 out of 44828
Found a safe distance color theory  => Similarity Score: 1.0
Completed 13192 out of 44828
Found sovereignty red tank!  => Similarity Score: 0.484848484848
Completed 13193 out of 44828
Found liberian girl - 2012 remaster michael jackson  => Similarity Score: 0.783783783784
Completed 13194 out of 44828
Found set the fire to the third bar snow patrol martha wainwright  => 

Found tell mama savoy brown  => Similarity Score: 1.0
Completed 13278 out of 44828
Found try honesty billy talent  => Similarity Score: 1.0
Completed 13279 out of 44828
Found that's entertainment the jam  => Similarity Score: 0.981818181818
Completed 13280 out of 44828
Completed 13281 out of 44828
Found here in your love (feat. hope darst) the belonging co hope darst  => Similarity Score: 0.246575342466
Completed 13282 out of 44828
Found why does it always rain on me? travis  => Similarity Score: 0.986301369863
Completed 13283 out of 44828
Found even the waves chroma key  => Similarity Score: 1.0
Completed 13284 out of 44828
Found dj - 1999 remastered version david bowie  => Similarity Score: 0.518518518519
Completed 13285 out of 44828
Found money on my mind sam smith  => Similarity Score: 1.0
Completed 13286 out of 44828
Found lonely is the night - 2010 - remaster billy squier  => Similarity Score: 0.780487804878
Completed 13287 out of 44828
Found made to love john legend  => Similari

TypeError: 'NoneType' object has no attribute '__getitem__'

In [39]:
for track in final_result:
    if score_result[track] > 0.80:
        db.cultural_tweet.update_many({"track_id": track},
                                {"$set" : {
                                    "duration_ms" : final_result[track]
                                }})
    


In [40]:
clean_dic = {track: final_result[track] for track in final_result if score_result[track] <= 0.80}

In [44]:
print("Songs with duration " + str(len(db.cultural_tweet.find({"duration_ms" : {"$exists" : True }}).distinct("track_id"))))
print("Songs without duration " + str(len(db.cultural_tweet.find({"duration_ms" : {"$exists" : False}}).distinct("track_id"))))
print("Songs in total " + str(len(db.cultural_tweet.distinct("track_id"))))


Songs with duration 12660
Songs without duration 38385
Songs in total 51045
